# TO DO

1) Add another check for the collision since now we are introducing a robot and not more a rigid material point moving in the plane

2) In the steer function add the trajectory controller taking in account the dynamics of the robot and the fact that the robot is controlled in velocity

3) Decied how to do the feedback

# CODE

In [1]:
#!pip install seaborn

In [1]:
import numpy as np
import math
from env import *
from plotting import *
import copy
from utils import *
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
class Node:
    def __init__(self, x, y):

        self.x = x
        self.xd = None
        self.xdd = None

        self.y = y
        self.yd = None
        self.ydd = None

        self.theta = None
        self.thetad = None
        self.thetadd = None

        self.parent = None
        self.cost = None

        self.commands = None
        self.conf = None


class RRT_STAR:
    def __init__(self, start, goal, env, step, prob_gs, n_iters):

        self.node_s = Node(start[0], start[1])
        self.node_g = Node(goal[0], goal[1])

        ##################### INITIAL CONDITION #################
        self.node_s.theta = theta
        self.node_s.xd = xd
        self.node_s.xdd = xdd
        self.node_s.yd = yd
        self.node_s.ydd = ydd
        self.node_s.thetad = thetad
        self.node_s.thetadd = thetadd
        self.node_s.conf = [(self.node_s.x, self.node_s.y)]
        self.node_g.conf = [(self.node_g.x, self.node_g.y)]

        #########################################################

        self.env = env
        self.n_iters = n_iters
        # CHECK IF START NODE AND GOAL NODE IN THE ENV......
        self.step = step
        self.prob_gs = prob_gs  # Empirical value
        self.utils = Utils()
        self.node_s.cost = 0
        self.Tree = [self.node_s]
        self.plotting = Plotting(self.node_s, self.node_g)
        # Empirical value
        self.radius = max(self.env.x_range[-1], self.env.y_range[-1])//10
        self.trav_map = np.clip(np.random.normal(3, 8, size=(50, 30)), 2, 100)
        self.max_vel = max_vel
        self.mass = mass
        self.a = a
        self.b = b
        self.mu = mu
        self.fr = fr
        self.Iz = Iz
        self.g = g
        self.d0 = d0
        self.r = r
        self.tau_max = tau_max
        self.t = t
        self.kv = kv
        self.kp = kp
        self.ka = ka

    def planning(self):
        ax = sns.heatmap(self.trav_map.T, linewidth=0.5)
        plt.show()
        for iter in range(self.n_iters):
            if iter % 500 == 0:
                print(iter)
            # print(iter)
            n_rand = self.sample()
            n_nearest, n_new = self.compute_NN(n_rand)

            if n_nearest.x == n_new.x and n_nearest.y == n_new.y:
                continue
            
            n_new, xd, xdd, yd, ydd, thetaf, thetad, thetadd, commands, conf = self.steer(
                n_nearest, n_new)

            if not self.utils.is_collision(n_nearest, n_new) and int(n_new.x) in range(0, self.env.x_range[-1]) and int(n_new.y) in range(0,self.env.y_range[-1]):
                # Assign to node velocity, acceleration and commands
                n_new.xd = xd
                n_new.xdd = xdd
                n_new.yd = yd
                n_new.ydd = ydd
                n_new.theta = thetaf
                n_new.thetad = thetad
                n_new.thetadd = thetadd
                n_new.commands = commands
                n_new.conf = conf

                N_near = self.compute_NN(n_new, near=True)
                n_min, cost_min = self.choose_parent(N_near, n_nearest, n_new)
                n_new.parent = n_min
                n_new.cost = cost_min
                # Insert Node
                self.Tree.append(n_new)
                self.rewire(N_near, n_min, n_new)
        path = [[self.node_g]]
        goal_state = self.get_goal_state()

        self.tot_path = self.compute_path(path, goal_state)

        self.plotting.animation(self.Tree, self.tot_path,
                                "rrt*, N = " + str(self.n_iters))

    def choose_parent(self, N_near, n_nearest, n_new):
        n_parent = n_nearest
        # Init. guess parent for sampled node
        cost_min = n_nearest.cost + self.compute_cost(n_nearest, n_new)
        
        # print('LEN: ', len(N_near))
        for n_near in N_near:
            # print('before_steer')
            if n_near.x == n_new.x and n_near.y == n_new.y:
                continue
            n_new_tr, xd, xdd, yd, ydd, thetaf, thetad, thetadd, commands, conf = self.steer(
                n_near, n_new)
            # print('after_steer')
            
            if not self.utils.is_collision(n_near, n_new_tr) and int(n_new_tr.x) in range(0, self.env.x_range[-1]) and int(n_new_tr.y) in range(0,self.env.y_range[-1]):

                inter_cost = n_near.cost + \
                    self.compute_cost(n_near, n_new_tr)  # Intermediate cost
                if n_new_tr.cost != None:
                    if inter_cost < n_new.cost and inter_cost < cost_min:
                        n_parent = n_near         # New best alternative
                        cost_min = inter_cost

                        n_new.xd = xd
                        n_new.xdd = xdd
                        n_new.yd = yd
                        n_new.ydd = ydd
                        n_new.theta = thetaf
                        n_new.thetad = thetad
                        n_new.thetadd = thetadd
                        n_new.commands = commands
                        n_new.conf = conf

                        n_new.x, n_new.y = n_new_tr.x, n_new_tr.y
                        n_new.x, n_new.y = n_new_tr.x, n_new_tr.y


                else:
                    if inter_cost < cost_min:
                        n_parent = n_near         # New best alternative
                        cost_min = inter_cost

                        n_new.xd = xd
                        n_new.xdd = xdd
                        n_new.yd = yd
                        n_new.ydd = ydd
                        n_new.theta = thetaf
                        n_new.thetad = thetad
                        n_new.thetadd = thetadd
                        n_new.commands = commands
                        n_new.conf = conf
                        
                        n_new.x, n_new.y = n_new_tr.x, n_new_tr.y
                        n_new.x, n_new.y = n_new_tr.x, n_new_tr.y
        # print('Sono uscito da Choose parent...')

        return n_parent, cost_min

    def rewire(self, N_near, n_parent, n_new):
        # print("LEN: ", len(N_near))
        for n_near in N_near:
            # print('kawabonga')
            if n_near.x == n_parent.x and n_near.y == n_parent.y:
                continue

            if n_near.x == n_new.x and n_near.y == n_new.y:
                continue

            z_near, xd, xdd, yd, ydd, thetaf, thetad, thetadd, commands, conf = self.steer(
                n_near, n_new)

            if not self.utils.is_collision(z_near, n_new) and int(z_near.x) in range(0, self.env.x_range[-1]) and int(z_near.y) in range(0,self.env.y_range[-1]):

                if n_new.cost + self.compute_cost(z_near, n_new) < n_near.cost:

                    n_near_old_idx = self.Tree.index(n_near)
                    z_near.parent = n_new
                    z_near.cost = n_new.cost + self.compute_cost(z_near, n_new)
                    self.Tree[n_near_old_idx] = z_near
                    
                    z_near.xd = xd
                    z_near.xdd = xdd
                    z_near.yd = yd
                    z_near.ydd = ydd
                    z_near.theta = thetaf
                    z_near.thetad = thetad
                    z_near.thetadd = thetadd
                    z_near.commands = commands
                    z_near.conf = conf
                    
        # print('sono uscito')
        return

    def compute_NN(self, n_curr, near=False):

        # CHECK COLLISION
        dist_list = np.array([self.compute_dist(n_curr, n_tree)
                             for n_tree in self.Tree])

        if not near:  # Nearest

            n_nearest = self.Tree[np.argmin(dist_list)]
            dist = min(np.min(dist_list), self.step)
            theta = math.atan2(n_curr.y-n_nearest.y, n_curr.x - n_nearest.x)
            n_new = Node(n_nearest.x + dist*math.cos(theta),
                         n_nearest.y + dist*math.sin(theta))

            return n_nearest, n_new
        else:        # Near
            node_list = [self.Tree[idx] for idx in range(
                len(dist_list)) if dist_list[idx] < self.radius]

            return node_list

    def sample(self):
        ''' Sample function RRT*, sample the goal node with a low probability '''
        if np.random.random() > self.prob_gs:
            x_new = np.random.uniform(self.env.x_range[-1])
            y_new = np.random.uniform(self.env.y_range[-1])
            # print(x_new, y_new)
            return Node(x_new, y_new)

        return self.node_g

    def get_goal_state(self):

        dist_list = np.array([self.compute_dist(node, self.node_g)
                             for node in self.Tree])
        min_idx = np.argmin(dist_list)
        goal_state = self.Tree[min_idx]

        return goal_state

    def compute_path(self, path, goal_state):

        n_curr = goal_state
        while n_curr.x != self.node_s.x and n_curr.y != self.node_s.y:
            path.append([n_curr])
            n_curr = n_curr.parent
        path.append([n_curr])

        return path

    def compute_dist(self, n1, n2):
        dist = math.sqrt((n1.x-n2.x)**2 + (n1.y-n2.y)**2)
        return dist

    def compute_cost(self, n1, n2):
        # Add Traversability!!!

        # Elevation on the real map
        n1_trav = self.trav_map[int(n1.x), int(n1.y)]
        n2_trav = self.trav_map[int(n2.x), int(n2.y)]

        increm = n2_trav - n1_trav  # Elevation between the two points
        plan_dist = self.compute_dist(n1, n2)  # Planar distance

        cost = math.sqrt(plan_dist**2 + increm**2)
        # cost = self.compute_dist(n1, n2)

        return cost

    def steer(self, n1, n2):
        '''I will consider the motion of the center of mass of the 4wdd using Runge Kutta integration'''
        x = n1.x
        y = n1.y
        theta = n1.theta

        xd = n1.xd
        xdd = n1.xdd

        yd = n1.yd
        ydd = n1.ydd

        thetad = n1.thetad
        thetadd = n1.thetadd

        dist = self.compute_dist(n1, n2)
        
        T = dist/self.max_vel
        T = math.ceil(T)

        z1, z2, z1d, z2d, z1dd, z2dd, z1ddd, z2ddd = self.compute_trajectory(
            n1, n2)

        qd0 = np.array([[xd*math.cos(theta) - yd*math.sin(theta)],
                       [xd*math.sin(theta) + yd*math.cos(theta)],
                       [thetad]])
        commands = []
        conf = []
        u0 = 0
        # print('Dist inside steer:', dist)
        # Ideally I want to go from n1 to n2 with the max velocity reachable by the robot
        for i in range(0, T*freq):
            i /= freq

            # The variable with the UNDERSCORE is the real one executed by the robot
            z1_, z2_, z1d_, z2d_, z1dd_, z2dd_ = self.compute_state(
                n1, xd, yd, theta, xdd, ydd, thetad, thetadd)

            r1 = z1ddd(i) + self.ka*(z1dd(i) - z1dd_) + self.kv * \
                (z1d(i) - z1d_) + self.kp*(z1(i) - z1_)
            r2 = z2ddd(i) + self.ka*(z2dd(i) - z2dd_) + self.kv * \
                (z2d(i) - z2d_) + self.kp*(z2(i) - z2_)

            Rx, Fy, Mr = self.compute_friction(xd, yd, thetad)

            eps_d = math.cos(theta)*r1 + math.sin(theta) * \
                r2 + 1/self.d0*xd*yd**2

            u1 = eps_d*1/freq + u0
            u0 = u1

            u2 = self.d0/xd*(math.sin(theta)*r1 -
                             math.cos(theta)*r2)-2/xd*u1*yd

            tau1 = self.r/2*(self.mass*u1 + self.mass/self.d0*yd**2+Rx)-self.r*self.d0/(
                2*self.t)*((self.mass+self.Iz/self.d0**2)*u2-self.mass/self.d0*xd*yd+Fy-Mr/self.d0)
            tau2 = self.r/2*(self.mass*u1 + self.mass/self.d0*yd**2+Rx)+self.r*self.d0/(
                2*self.t)*((self.mass+self.Iz/self.d0**2)*u2-self.mass/self.d0*xd*yd+Fy-Mr/self.d0)

            if tau1 > self.tau_max or tau1 < - self.tau_max:
                tau1 = np.sign(tau1)*self.tau_max
            if tau2 > self.tau_max or tau2 < - self.tau_max:
                tau2 = np.sign(tau2)*self.tau_max

            v, w, qdf, xd, yd, xdd, ydd, thetad, thetadd = self.compute_control(
                tau1, tau2, Rx, Fy, Mr, theta, qd0)
            qd0 = qdf

            commands.append((v, w))

            xf, yf, thetaf = self.RK_integration(x, y, theta, T, v, w)
            conf.append((xf, yf))
            
            theta = thetaf

        n_new = Node(xf, yf)

        return n_new, xd, xdd, yd, ydd, thetaf, thetad, thetadd, commands, conf

    def RK_integration(self, x, y, theta, T, v, w):
        ''' Runge Kutta integration for a generic differential drive robot'''
        xf = x+v*T*math.cos(theta*(w*T)/2)
        yf = y+v*T*math.sin(theta*(w*T)/2)
        thetaf = theta+w*T

        return xf, yf, thetaf

    def compute_trajectory(self, n1, n2):
        ''' In this function I assume that the robot is moving in a straight line from a node to the next one'''
        z1 = np.poly1d([1, 0, 0, 0]) + n1.x  # third degree polynomial

        const = float((n2.y - n1.y)/(n2.x - n1.x + 1e-6))  # line equation

        z2 = n1.y + const*(z1 - n1.x)

        z1d = z1.deriv()
        z2d = const*z1d

        z1dd = z1d.deriv()
        z2dd = const*z1dd

        z1ddd = z1dd.deriv()
        z2ddd = const*z1ddd

        return z1, z2, z1d, z2d, z1dd, z2dd, z1ddd, z2ddd

    def compute_friction(self, xd, yd, thetad):
        '''This function computes the resistive force on the x axis Rx, the viscous friction on the y axis Fy and the resistive moment Mr'''
        x1d = x4d = xd - self.t*thetad
        x3d = x2d = xd + self.t*thetad

        y1d = y2d = yd + self.a*thetad

        y3d = y4d = yd - self.b*thetad

        Rx = self.fr*(self.mass*self.g)/2 * (np.sign(x1d) + np.sign(x2d))
        Fy = self.mu*((self.mass*self.g)/(self.a + self.b)) * \
            (self.b*np.sign(y1d) + self.a*np.sign(y3d))
        Mr = self.mu*((self.a*self.b*self.mass*self.g)/(self.a + self.b))*(np.sign(y1d) -
                                                                           np.sign(y3d)) + self.fr*self.t*self.mass*self.g/2*(np.sign(x2d) - np.sign(x1d))
        return Rx, Fy, Mr

    def compute_control(self, tau1, tau2, Rx, Fy, Mr, theta, qd0):
        '''This function computes the commands v and w useful for the Runge Kutta integration '''
        M = np.array([[self.mass, 0, 0], [0, self.mass, 0], [0, 0, self.Iz]])
        c = np.array([[Rx*math.cos(theta)-Fy*math.sin(theta)],
                     [Rx*math.sin(theta)+Fy*math.cos(theta)],
                      [Mr]])
        E = np.array([[math.cos(theta)/self.r, math.cos(theta)/self.r],
                      [math.sin(theta)/self.r, math.sin(theta)/self.r],
                      [self.t/self.r, -self.t/self.r]])
        tau = np.array([[tau1], [tau2]])

        R = np.array([[math.cos(theta), -math.sin(theta), 0],
                      [math.sin(theta), math.cos(theta), 0],
                      [0, 0, 1]])
        
        qdd = np.linalg.inv(M)@(E@tau - c)

        qd = qdd*1/freq + qd0

        vel = R.T@qd
        xd = vel[0]
        yd = vel[1]

        acc = R.T@qdd
        xdd = acc[0]
        ydd = acc[1]
        thetadd = qdd[2, 0]
        thetad = qd[2, 0]

        v = math.sqrt(qd[0]**2 + qd[1]**2)
        w = (qdd[1]*qd[0] - qdd[0]*qd[1]) / (qd[0]**2 + qd[1]**2)

        return float(v), float(w[0]), float(qd[0]), float(xd[0]), float(yd[0]), float(xdd[0]), float(ydd[0]), float(thetad), float(thetadd)

    def compute_state(self, n1, xd, yd, theta, xdd, ydd, thetad, thetadd):
        z1_ = n1.x
        z2_ = n1.y

        z1d_ = xd*math.cos(theta) - yd*math.sin(theta) - \
            self.d0*math.sin(theta)*thetad
        z2d_ = xd*math.sin(theta) + yd*math.cos(theta) + \
            self.d0*math.cos(theta)*thetad

        z1dd_ = xdd*math.cos(theta) - yd*thetad*math.sin(theta) - self.d0 * \
            math.cos(theta)*thetad**2 - self.d0*math.sin(theta)*thetadd

        z2dd_ = ydd*math.sin(theta) + xd*thetad*math.cos(theta) - self.d0 * \
            math.sin(theta)*thetad**2 + self.d0*math.cos(theta)*thetadd
        return z1_, z2_, z1d_, z2d_, z1dd_, z2dd_


In [3]:
env = Env(x_bounds=x_bounds, y_bounds=y_bounds)

rrt = RRT_STAR(start, goal, env, step, prob_gs, n_iters)

In [ ]:
rrt.planning()

In [5]:
N_near = [n_nearest, n1, n2]

NameError: name 'n_nearest' is not defined

In [ ]:
rrt.Tree

In [ ]:
def get_key(el):

    return el[0]

l = [(30, 1), (20, 2), (5, 4), (10,11)]
idx_min(l, key = get_key)


(5, 4)

In [ ]:
rrt.rewire(N_near, n1, n_new)

In [ ]:
rrt.Tree[1].cost

In [89]:
np.clip(np.random.normal(3, 1, size = (30, 50)), 0, 10)

array([[2.77952685, 2.34587721, 2.58542192, ..., 3.46067756, 2.41679018,
        4.44843464],
       [3.29843258, 2.9535215 , 2.71760168, ..., 2.38485116, 1.67587108,
        2.96043352],
       [2.10235456, 2.63892518, 1.38002193, ..., 3.76144618, 0.69321575,
        0.24621384],
       ...,
       [4.96680313, 2.81058999, 2.36895317, ..., 3.19331032, 3.48330797,
        3.12140231],
       [4.05210039, 4.72569205, 3.704484  , ..., 5.03792807, 4.028809  ,
        1.33753239],
       [2.85725463, 2.98661474, 2.77125505, ..., 2.95868284, 1.90177008,
        1.65398131]])